In [0]:
# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

In [1]:
!nvidia-smi

Tue May 19 10:38:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import torchvision
import scipy.ndimage
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2ycbcr
from scipy import misc
from PIL import Image
import numpy as np
import cv2, os, math, random, json, time, pickle, imageio 
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool

%matplotlib inline

In [4]:
cuda = torch.device('cuda:0')  # CUDA GPU 0
print(torch.cuda.get_device_name(cuda))
print(torch.__version__)

Tesla P100-PCIE-16GB
1.5.0+cu101


In [0]:
data_root = "/content/drive/My Drive/SML Project/Dataset/"
data_paths = {"DIV2K": {"train":
                      {"HR":"DIV2K/DIV2K_train_HR/",
                      "LR_BI": { "X2":"DIV2K/DIV2K_train_LR_bicubic/X2/", "X3":"DIV2K/DIV2K_train_LR_bicubic/X3/", "X4":"DIV2K/DIV2K_train_LR_bicubic/X4/" },
                      "LR_UN": { "X2":"DIV2K/DIV2K_train_LR_unknown/X2/", "X3":"DIV2K/DIV2K_train_LR_unknown/X3/", "X4":"DIV2K/DIV2K_train_LR_unknown/X4/" }
                      } ,
                        "val":
                      {"HR":"DIV2K/DIV2K_valid_HR/",
                      "LR_BI": { "X2":"DIV2K/DIV2K_valid_LR_bicubic/X2/", "X3":"DIV2K/DIV2K_valid_LR_bicubic/X3/", "X4":"DIV2K/DIV2K_valid_LR_bicubic/X4/" },
                      "LR_UN": { "X2":"DIV2K/DIV2K_valid_LR_unknown/X2/", "X3":"DIV2K/DIV2K_valid_LR_unknown/X3/", "X4":"DIV2K/DIV2K_valid_LR_unknown/X4/" }
                      } },
              "Flickr2K":{"train":
                         {  "HR":"Flickr2K/Flickr2K_HR/",
                             "LR_BI":  { "X2":"Flickr2K/Flickr2K_LR_bicubic/X2/", "X3":"Flickr2K/Flickr2K_LR_bicubic/X3/", "X4":"Flickr2K/Flickr2K_LR_bicubic/X4/" },
                             "LR_UN":  { "X2":"Flickr2K/Flickr2K_LR_unknown/X2/", "X3":"Flickr2K/Flickr2K_LR_unknown/X3/", "X4":"Flickr2K/Flickr2K_LR_unknown/X4/" }
                         } ,
                          "val":
                        {"HR":"DIV2K/DIV2K_valid_HR/",
                        "LR_BI": { "X2":"DIV2K/DIV2K_valid_LR_bicubic/X2/", "X3":"DIV2K/DIV2K_valid_LR_bicubic/X3/", "X4":"DIV2K/DIV2K_valid_LR_bicubic/X4/" },
                        "LR_UN": { "X2":"DIV2K/DIV2K_valid_LR_unknown/X2/", "X3":"DIV2K/DIV2K_valid_LR_unknown/X3/", "X4":"DIV2K/DIV2K_valid_LR_unknown/X4/" }
                      } },
                "Set5":{"HR":  {"X2":"Set5/HR/x2/", "X3":"Set5/HR/x3/", "X4":"Set5/HR/x4/" },
                        "LR_BI": { "X2":"Set5/LR/LR_BI/x2/", "X3":"Set5/LR/LR_BI/x3/", "X4":"Set5/LR/LR_BI/x4/" },
                      },
                "Set14":{"HR":  {"X2":"Set14/HR/X2/", "X3":"Set14/HR/X3/", "X4":"Set14/HR/X4/" },
                        "LR_BI": { "X2":"Set14/LR/LR_BI/X2/", "X3":"Set14/LR/LR_BI/X3/", "X4":"Set14/LR/LR_BI/X4/" },
                      },
              }

kernel_attrs = {"2":(6,2,2),"3":(7,3,2),"4":(8,4,2)}
patch_sizes = {"2":60,"3":50,"4":40}

In [0]:
class AutoAugment:
    def __init__(self,dataset = 'DIV2K', split = 'train', downsampling = "LR_BI", scale = 4):
        self.split = split
        self.dataset = dataset
        self.downsampling = downsampling
        self.scale = scale
        self.zooms = [0.8,0.7,0.6,0.5]

        self.hr,self.lr = self._get_img_list()

    def _aug(self,img,size,ctr,path):
        h,w,_ = img.shape
        img = np.array(Image.fromarray(img).resize((int(w*size),int(h*size)),resample=Image.BICUBIC))
        h,w,_ = img.shape
        h = h - np.mod(h, self.scale)
        w = w - np.mod(w, self.scale)
        img = img[0:h, 0:w, :]

        img_flip = np.array(Image.fromarray(img).rotate(180,resample=Image.BICUBIC))
        lr = np.array(Image.fromarray(img).resize((int(w/int(self.scale)),int(h/self.scale)),resample=Image.BICUBIC))
        lr_flip = np.array(Image.fromarray(img_flip).resize((int(w/self.scale),int(h/self.scale)),resample=Image.BICUBIC)) 

        imageio.imwrite(path.split('.')[0]+"_"+str(ctr)+".png" , img)
        imageio.imwrite(path.split('.')[0]+"_"+str(ctr+1)+".png" , img_flip)
        imageio.imwrite(data_root+data_paths[self.dataset][self.split][self.downsampling]["X"+str(self.scale)]+path.split('/')[-1].split('.')[0]+"x"+str(self.scale)+"_"+str(ctr)+".png", lr)
        imageio.imwrite(data_root+data_paths[self.dataset][self.split][self.downsampling]["X"+str(self.scale)]+path.split('/')[-1].split('.')[0]+"x"+str(self.scale)+"_"+str(ctr+1)+".png", lr_flip)

    def _aug_img(self,path):
        print("Augmenting",path.split('/')[-1].split('.')[0],"/ 800")
        img = imageio.imread(path)
        ctr = 9 #
        for sz in self.zooms:
            self._aug(img,sz,ctr,path)
            ctr+=2
        h,w,_ = img.shape
        h = h - np.mod(h, self.scale)
        w = w - np.mod(w, self.scale)
        img = img[0:h, 0:w, :]
        img_flip = np.array(Image.fromarray(img).rotate(180,resample=Image.BICUBIC))
        imageio.imwrite(path.split('.')[0]+"_"+str(ctr)+".png" , img_flip)
        lr_flip = np.array(Image.fromarray(img_flip).resize((int(w/self.scale),int(h/self.scale)),resample=Image.BICUBIC)) 
        imageio.imwrite(data_root+data_paths[self.dataset][self.split][self.downsampling]["X"+str(self.scale)]+path.split('/')[-1].split('.')[0]+"x"+str(self.scale)+"_"+str(ctr)+".png", lr_flip)     

    def autoaugment(self):
        if len(self.hr)!=800:
            print(len(self.hr))
            return
        # pool = ThreadPool()
        # pool.map(self._aug_img, self.hr)
        # pool.close()
        # pool.join()
        for pth in self.hr:
            if "_" not in pth.split("/")[-1]:
                self._aug_img(pth)
            

    def cleanup(self):
        if len(self.hr) == 800:
            print("Nothing to cleanup")
            return
        for pth in self.hr:
            if "_" in pth.split("/")[-1]:
                os.remove(pth)
                
    def _get_img_list(self):
        hr_imgs = [data_root+data_paths[self.dataset][self.split]["HR"]+img for img in os.listdir(data_root+data_paths[self.dataset][self.split]["HR"])]
        lr_imgs = [data_root+data_paths[self.dataset][self.split][self.downsampling]["X"+str(self.scale)]+img for img in os.listdir(data_root+data_paths[self.dataset][self.split][self.downsampling]["X"+str(self.scale)])] 
        
        hr_imgs.sort()
        lr_imgs.sort()
        return hr_imgs,lr_imgs

In [0]:
# aug = AutoAugment()
# aug.autoaugment()
# aug.cleanup()

In [0]:
class DataSet(Dataset):
    
    def __init__(self,dataset = "train", split = 'train', downsampling = "LR_BI", scale = "X4", no_aug = False):
        
        self.split = split
        self.dataset = ["DIV2K"] if dataset == "train" else [dataset] # ,"Flickr2K"
        self.downsampling = downsampling
        self.scale = scale
        self.no_aug = no_aug

        random.seed(1)
        
        if self.split == 'train':
            self.start_percent = 0
            self.end_percent = 0.8
        elif self.split == 'val':
            self.start_percent = 0.8
            self.end_percent = 0.9
        elif self.split == 'test':
            self.start_percent = 0.9
            self.end_percent = 1
            
            
        self.hr,self.lr = self._get_img_list() 
        self.hr,self.lr = self.hr,self.lr

        self.transforms = transforms.Compose([ transforms.ToPILImage(),  
                                               transforms.ToTensor()])                   
    def __len__(self):
        return len(self.hr)
    
    def __getitem__(self,idx):
        lr, hr = self._get_input_tensor(idx)
        return lr, hr ,self.hr[idx]

    def _augment(self,lr,hr, prob=0.5):
        augmented_lr = lr
        augmented_hr = hr
        x = random.uniform(0,1)
        if x < prob:
            augmented_lr = augmented_lr.transpose(1,0,2)
            augmented_hr = augmented_hr.transpose(1,0,2)
        if x < prob:
            augmented_lr = augmented_lr.transpose(1,0,2)[::-1,::-1,:]
            augmented_hr = augmented_hr.transpose(1,0,2)[::-1,::-1,:]
        if x < prob:
            augmented_lr = np.fliplr(augmented_lr)
            augmented_hr = np.fliplr(augmented_hr)
        return augmented_lr, augmented_hr

    def _extract_patch(self,lr,hr):
        lh,lw,_ = lr.shape
        hh,hw,_ = hr.shape

        scale = int(self.scale[-1])
        patch = patch_sizes[str(scale)]

        while True:
          ld = int(min(lh,lw))
          lhp = np.random.randint(10,ld-patch-10)
          lwp = np.random.randint(10,ld-patch-10)

          lr_crop = lr[lhp:(lhp+patch),lwp:(lwp+patch),:]
          hr_crop = hr[lhp*scale:(lhp+patch)*scale,lwp*scale:(lwp+patch)*scale,:]

          try:
            assert hr_crop.shape[0]==hr_crop.shape[1], hr_crop.shape
            break
          except:
            lh,lw = lw,lh
            continue

        return lr_crop, hr_crop

    def _get_input_tensor(self,idx):    
        # st = time.time()
        lr_im = cv2.imread(self.lr[idx],1)
        hr_im = cv2.imread(self.hr[idx],1) 
        lr_im = cv2.cvtColor(lr_im,cv2.COLOR_BGR2RGB)
        hr_im = cv2.cvtColor(hr_im,cv2.COLOR_BGR2RGB)
        if self.split == "train":
          lr_im,hr_im = self._extract_patch(lr_im,hr_im)
        # print(lr_im.shape,hr_im.shape)
        # lr_im, hr_im = self._augment(lr_im,hr_im,0.5)
        lr_im = self.transforms(lr_im)
        hr_im = self.transforms(hr_im)
        # print(time.time()-st)
        return lr_im, hr_im
  
    def _get_img_list(self):

        hr, lr = [], []
        for dataset in self.dataset:
            if self.split=="val":
                hr_imgs = [data_root+data_paths[dataset]["HR"][self.scale]+img for img in os.listdir(data_root+data_paths[dataset]["HR"][self.scale])]
                lr_imgs = [data_root+data_paths[dataset]["LR_BI"][self.scale]+img for img in os.listdir(data_root+data_paths[dataset]["LR_BI"][self.scale])]
            else:
                hr_imgs = [data_root+data_paths[dataset][self.split]["HR"]+img for img in os.listdir(data_root+data_paths[dataset][self.split]["HR"])  if  not (self.no_aug and "_" in img) ]
                lr_imgs = [data_root+data_paths[dataset][self.split][self.downsampling][self.scale]+img for img in os.listdir(data_root+data_paths[dataset][self.split][self.downsampling][self.scale]) if  not (self.no_aug and "_" in img) ] 
                
            hr_imgs.sort()
            lr_imgs.sort()
            
            if dataset == "Flickr2K":
                hr_imgs = hr_imgs[int(self.start_percent*len(hr_imgs)):int(self.end_percent*len(hr_imgs))]
                lr_imgs = lr_imgs[int(self.start_percent*len(lr_imgs)):int(self.end_percent*len(lr_imgs))]

            hr.extend(hr_imgs)
            lr.extend(lr_imgs)

        return hr, lr


In [0]:
# d = DataSet('train','train',scale="X3",no_aug=True)
# for i in range(50):
#   print(d[i][0].shape,d[i][1].shape)

In [0]:
class MeanShift(nn.Conv2d):
    def __init__(self, rgb_mean, rgb_std, sign=-1):
        super(MeanShift, self).__init__(3, 3, kernel_size=1)
        std = torch.Tensor(rgb_std)
        self.weight.data = torch.eye(3).view(3, 3, 1, 1)
        self.weight.data.div_(std.view(3, 1, 1, 1))
        self.bias.data = sign * 255. * torch.Tensor(rgb_mean)
        self.bias.data.div_(std)
        for p in self.parameters():
            p.requires_grad = False

In [0]:
class S_E_Block(nn.Module):
    def __init__(self, in_ch, r):
        super(S_E_Block, self).__init__()
        self.linear_1 = nn.Linear(in_ch, in_ch//r)
        self.linear_2 = nn.Linear(in_ch//r, in_ch)
        self.sigmoid = nn.Sigmoid()

        for m in self.modules():
          if isinstance(m, nn.Linear):
              nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
              if m.bias is not None:
                  m.bias.data.zero_()

    def forward(self, ip):
        x = ip
        x = x.view((x.shape[0],x.shape[1],-1)).mean(-1)
        x = F.relu(self.linear_1(x), inplace=True)
        x = self.linear_2(x)
        x = x.unsqueeze(-1).unsqueeze(-1)
        x = self.sigmoid(x)
        x = ip * x
        return x

In [0]:

class LRFeatureExtractor(nn.Module):

    def __init__(self,in_channels = 3, num_filters = 32):
        super(LRFeatureExtractor, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, num_filters*4, 3 , stride=1, padding=1, dilation=1, bias=True)
        self.activation = nn.PReLU(num_parameters=1, init=0.2)
        self.conv2 = nn.Conv2d(num_filters*4, num_filters, 1 , stride=1, padding=0, dilation=1, bias=True)
        
        for m in self.modules():
          if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
              nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
              if m.bias is not None:
                  m.bias.data.zero_()


    def forward(self,x):
      x = self.activation(self.conv1(x))
      x = self.activation(self.conv2(x))
      return x

In [0]:
class ProjectionBlock(nn.Module):

    def __init__(self, block_num, num_filters, kernel_size, stride, padding):
        super(ProjectionBlock, self).__init__()
        self.block_num = block_num
        if block_num > 1:
            self.compress_deconv = nn.Conv2d(block_num*num_filters, num_filters, 1)
            self.compress_conv = nn.Conv2d(block_num*num_filters, num_filters, 1)

        self.deconv = nn.ConvTranspose2d(num_filters, num_filters, kernel_size, stride = stride, padding = padding)
        self.conv = nn.Conv2d(num_filters, num_filters, kernel_size, stride = stride, padding = padding)
        self.activation = nn.PReLU(num_parameters=1, init=0.2)

        self.se1 = S_E_Block(in_ch=num_filters, r=16)
        self.se2 = S_E_Block(in_ch=num_filters, r=16)

        for m in self.modules():
          if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
              nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
              if m.bias is not None:
                  m.bias.data.zero_()

    def forward(self,lr,hr):
        if self.block_num > 1:
            h = self.activation(self.se1(self.deconv(self.activation(self.compress_deconv(lr)))))
            h = torch.cat((h,hr), dim=1)
            l = self.activation(self.se2(self.conv(self.activation(self.compress_conv(h)))))
        else:
            h = self.activation(self.se1(self.deconv(lr)))
            l = self.activation(self.se2(self.conv(h)))
        l = torch.cat((l,lr), dim=1)
        return l, h

In [0]:
class FeedbackBlock(nn.Module):

    def  __init__(self, num_blocks, num_filters, kernel_size, stride, padding):
        super(FeedbackBlock, self).__init__()
        self.num_filters = num_filters
        self.num_blocks = num_blocks

        self.lr_in_compress = nn.Conv2d(num_filters*2, num_filters, 1 , stride=1, padding=0, dilation=1, bias=True)
        self.blocks = [ProjectionBlock(1,num_filters, kernel_size, stride, padding)]
        for blk_num in range(2,num_blocks+1):
            self.blocks.append(ProjectionBlock(blk_num,num_filters, kernel_size, stride, padding))
        self.lr_out_compress = nn.Conv2d(num_filters*num_blocks, num_filters, 1 , stride=1, padding=0, dilation=1, bias=True)
        self.activation = nn.PReLU(num_parameters=1, init=0.2)

        for m in self.modules():
          if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
              nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
              if m.bias is not None:
                  m.bias.data.zero_()

        for blk in self.blocks:
          blk.cuda()
          
    def forward(self,x):
        lr = self.activation(self.lr_in_compress(x))
        lr, hr = self.blocks[0](lr,0)
        del x
        for i in range(1,self.num_blocks):
            lr, hr = self.blocks[i](lr, hr) 
        del hr
        lr = lr[:,self.num_filters:,:,:]
        lr = self.activation(self.lr_out_compress(lr))
        return lr

In [0]:
class SRFBN_Orig(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, num_features=32, num_steps=4, num_groups=6, upscale_factor=4, act_type = 'prelu', norm_type = None):
        super(SRFBN_Orig, self).__init__()

        kernel_size, stride, padding = kernel_attrs[str(upscale_factor)]

        self.num_steps = num_steps
        self.num_features = num_features
        self.upscale_factor = upscale_factor

        rgb_mean = (0.4488, 0.4371, 0.4040)
        rgb_std = (1.0, 1.0, 1.0)
        self.sub_mean = MeanShift(rgb_mean, rgb_std)

        # LR feature extraction block
        self.lrfb = LRFeatureExtractor(3,num_features)

        # basic block
        self.block = FeedbackBlock(num_groups, num_features, kernel_size, stride, padding)

        self.rb = nn.Sequential(nn.ConvTranspose2d(num_features, num_features, kernel_size, stride = stride, padding = padding),
                              S_E_Block(in_ch=num_features, r=16),
                              nn.PReLU(num_parameters=1, init=0.2),
                              nn.Conv2d(num_features,out_channels,3,padding=1,bias = True))
 
        self.add_mean = MeanShift(rgb_mean, rgb_std, 1)

        for m in self.modules():
          classname = m.__class__.__name__
          if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
              if classname != "MeanShift":
                  nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
                  if m.bias is not None:
                      m.bias.data.zero_()

    def forward(self, x):
        x = self.sub_mean(x)
        inter_res = nn.functional.interpolate(x, scale_factor=self.upscale_factor, mode='bilinear', align_corners=False)
        x = self.lrfb(x)

        f_in = torch.cat((x,x), dim=1)
        outs = []
        for it in range(self.num_steps):
            f_out = self.block(f_in)
            f_in = torch.cat((f_in[:,self.num_features:,:,:],f_out), dim=1)
            outs.append(self.add_mean(torch.add(self.rb(f_out),inter_res)))

        return outs # return output of every timesteps


In [0]:
class Trainer:

  def __init__(self, batch_size = 16, scale = 4, num_filters = 32, num_blocks = 6, num_iters = 4, skip_train = False):
      self.batch_size = batch_size
      self.scale = scale
      self.num_filters = num_filters
      self.num_blocks = num_blocks
      self.num_iters = num_iters
      self.skip_train = skip_train
      self._create_dataloaders()
      self._create_model()
      self._create_peripherals()

  def _create_dataloaders(self):
      if not self.skip_train:
        self.trainset = DataSet(dataset = 'train', split = 'train', downsampling = "LR_BI", scale = "X"+str(self.scale), no_aug=True)
        self.trainloader = DataLoader(self.trainset, batch_size=self.batch_size, shuffle=True, drop_last=False, num_workers=4)
      # self.valset = DataSet(dataset = 'Set5', split = 'val', downsampling = "LR_BI", scale = "X"+str(self.scale))
      # self.valloader = DataLoader(self.valset, batch_size=1, shuffle=False, drop_last=False)
      self.valset = DataSet(dataset = 'Set14', split = 'val', downsampling = "LR_BI", scale = "X"+str(self.scale))
      self.valloader = DataLoader(self.valset, batch_size=1, shuffle=False, drop_last=False)

  def _create_model(self):
      # self.model = SRFBN(num_blocks=self.num_blocks, num_iters=self.num_iters, scale=self.scale, num_filters = self.num_filters)
      self.model = SRFBN_Orig(upscale_factor = self.scale)

  def _create_peripherals(self):
      self.last_batch_loss = float('inf')
      self.best_ssim = -1
      self.start_epoch = 1

  def load_checkpoint(self):
      self.start_epoch = 1
      ckpt_path = "/content/drive/My Drive/SML Project/checkpoints/SRFBN_SE_"+str(self.scale)+"_best"+".pth"   # 
      if os.path.exists(ckpt_path):
          print("Loading from checkpoint...")
          checkpoint = torch.load(ckpt_path)
          self.model.load_state_dict(checkpoint['model'])
          self.optimizer.load_state_dict(checkpoint['optimizer'])
          self.scheduler.load_state_dict(checkpoint['scheduler'])
          self.start_epoch = checkpoint['epoch']+1
          self.best_ssim = checkpoint['ssim']
          self.record = checkpoint['record']

  def save_checkpoint(self,epoch,best = False):
      print("Saving checkpoint...")
      torch.save({
          'epoch': epoch,
          'model': self.model.state_dict(),
          'optimizer': self.optimizer.state_dict(),
          'scheduler': self.scheduler.state_dict(),
          'ssim': self.best_ssim,
          'record': self.record
          }, "/content/drive/My Drive/SML Project/checkpoints/SRFBN_SE_"+str(self.scale)+".pth")
      if best:
          print("Saving best checkpoint...")
          torch.save({
              'epoch': epoch,
              'model': self.model.state_dict(),
              'optimizer': self.optimizer.state_dict(),
              'scheduler': self.scheduler.state_dict(),
              'ssim': self.best_ssim,
              'record': self.record
              }, "/content/drive/My Drive/SML Project/checkpoints/SRFBN_SE_"+str(self.scale)+"_best"+".pth")
          
  def logger(self):
      plt.plot(self.record['validation_losses'])
      plt.plot(self.record['epoch_losses'])
      plt.suptitle('Losses')
      plt.savefig("/content/drive/My Drive/SML Project/plots/Training_SR/Log/SRFBN_SE_"+str(self.scale)+"_loss.png",dpi=1200)
      plt.close()
      plt.plot(self.record['validation_psnr'])
      plt.suptitle('PSNR')
      plt.savefig("/content/drive/My Drive/SML Project/plots/Training_SR/Log/SRFBN_SE_"+str(self.scale)+"_psnr.png",dpi=1200)
      plt.close()
      plt.plot(self.record['validation_ssim'])
      plt.suptitle('SSIM')
      plt.savefig("/content/drive/My Drive/SML Project/plots/Training_SR/Log/SRFBN_SE_"+str(self.scale)+"_ssim.png",dpi=1200)
      plt.close()
      
  def PSNR(self,img1, img2, max_val=225):
      # assert img1.shape == img2.shape
      # if len(img1.shape) >= 3:
      #     img1 = rgb2ycbcr(img1)
      #     img2 = rgb2ycbcr(img2)
      mse = img1 - img2
      mse = np.mean(mse ** 2)
      psnr = 10 * math.log((max_val ** 2 )/ mse, 10)
      return psnr

  def SSIM(self,img1, img2):
      return ssim(img1, img2, multichannel=False) # True

  def get_metrics(self, sr_batch, hr_batch):
      ssim_s = []
      psnr_s = []
      for ind in range(sr_batch.shape[0]):
          sr, hr = imgYCC = cv2.cvtColor(sr_batch[ind,:,:,:], cv2.COLOR_RGB2YCR_CB), cv2.cvtColor(hr_batch[ind,:,:,:], cv2.COLOR_RGB2YCR_CB)
          ssim_s.append(self.SSIM(sr[:,:,0],hr[:,:,0]))
          psnr_s.append(self.PSNR(sr[:,:,0],hr[:,:,0]))
      ssim_s = sum(ssim_s)/len(ssim_s)
      psnr_s = sum(psnr_s)/len(psnr_s)
      return ssim_s , psnr_s

  def train(self):
      os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

      self.record={'epoch_losses':[],'validation_losses':[],'validation_psnr':[],'validation_ssim':[]}
  
      pix_loss = nn.L1Loss()
      pix_loss = pix_loss.cuda()
      torch.cuda.empty_cache()
      self.model = self.model.to(cuda)
      
      self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001 , weight_decay=0)
      self.scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer,[70,140,200,250],0.5)          
      self.load_checkpoint()
      print("Starting from epoch = ",self.start_epoch)

      for epoch in range(self.start_epoch,self.start_epoch+1000):

          if not self.skip_train:

            self.model.train()         
            batch_losses = []
            
            for i, batch in enumerate(self.trainloader):
                self.optimizer.zero_grad()
                
                lr,hr = batch[0],batch[1]  
                lr = lr.to(cuda)
                hr = hr.to(cuda)   
                
                sr_outs = self.model.forward(lr) 
                out_loss = 0
                for sr in sr_outs:
                  # print(sr.detach().cpu().numpy().shape,hr.detach().cpu().numpy().shape)
                  out_loss += pix_loss(sr,hr)
                
                out_loss.backward()

                if (i+1)%5 == 0:
                    print("Iterations completed = ",i,"/",len(self.trainloader))

                if out_loss < 3* self.last_batch_loss:
                    self.optimizer.step()
                    self.last_batch_loss = out_loss
                
                batch_losses.append(out_loss.item())
                    
            self.record['epoch_losses'].append(sum(batch_losses)/len(batch_losses))
            print("After epoch ",epoch," Training Loss = ",self.record['epoch_losses'][-1])

          self.optimizer.zero_grad()
          self.model.eval()
          batch_losses = []
          batch_psnr = []
          batch_ssim = []

          for i, batch in enumerate(self.valloader):
              lr,hr = batch[0],batch[1]  
              lr = lr.to(cuda)
              hr = hr.to(cuda)   
              
              sr_outs = self.model.forward(lr)
              out_loss = 0
              for sr in sr_outs:
                out_loss += pix_loss(sr,hr)
              
              out_loss.backward()
              batch_losses.append(out_loss.item())
              sr_ip, hr_ip = sr_outs[-1].detach().cpu().permute(0, 2, 3, 1).numpy(), hr.detach().cpu().permute(0, 2, 3, 1).numpy()
              ssim, psnr = self.get_metrics(sr_ip, hr_ip)
              batch_psnr.append(psnr)
              batch_ssim.append(ssim)
              
              # plt.imshow(np.hstack((sr_ip[0,:,:,:],hr_ip[0,:,:,:])))
              # plt.savefig("/content/drive/My Drive/SML Project/plots/Training_SR/X"+str(self.scale)+"-SE/"+str(epoch)+"_"+str(i)+".png",dpi=1200)
              # plt.close()
             
          
          self.record['validation_psnr'].append(sum(batch_psnr)/len(batch_psnr))
          self.record['validation_ssim'].append(sum(batch_ssim)/len(batch_ssim))
          self.record['validation_losses'].append(sum(batch_losses)/len(batch_losses))

          print("After epoch ",epoch," Val Loss = ",self.record['validation_losses'][-1]," Val SSIM = ",self.record['validation_ssim'][-1]," Val PSNR = ",self.record['validation_psnr'][-1])
          self.scheduler.step()
          break
          last_best = self.best_ssim
          if self.record['validation_ssim'][-1]>self.best_ssim:
              self.best_ssim = self.record['validation_ssim'][-1]
          self.save_checkpoint(epoch,self.record['validation_ssim'][-1]>last_best)

          self.logger()
          

In [29]:
trainer = Trainer(batch_size=16,scale=3,skip_train=False) # batch_size = 30 for scale 2

OSError: ignored

In [0]:
trainer.train() 

In [0]:
import numpy as np
import cv2
import random


def augment(image, ratio_noraml_anticlock_clock_fliphr):
    ratio_noraml_anticlock_clock_fliphr = ratio_noraml_anticlock_clock_fliphr/np.sum(ratio_noraml_anticlock_clock_fliphr)
    ratio_noraml_anticlock_clock_fliphr = [sum(ratio_noraml_anticlock_clock_fliphr[:x]) for x in range(1, len(ratio_noraml_anticlock_clock_fliphr)+1)]
    augmented_image = None
    x = random.uniform(0,1)
    if x < ratio_noraml_anticlock_clock_fliphr[0]:
        augmented_image = image
    elif x < ratio_noraml_anticlock_clock_fliphr[1]:
        augmented_image = np.rot90(image, 1)
    elif x < ratio_noraml_anticlock_clock_fliphr[2]:
        augmented_image = np.rot90(image, 3)
    elif x < ratio_noraml_anticlock_clock_fliphr[3]:
        augmented_image = np.fliplr(image)
    return augmented_image


def BD(image, size):
    """
    Gaussian blur with kernel 7*7 with stddev 1.6 followed by  down sampling with BICUBIC interpolation
    :param image:
    :return:
    """
    blurred_image = cv2.GaussianBlur(image, (7, 7), 1.6)
    HR_img =cv2.resize(blurred_image, (int(blurred_image.shape[1]*size/100), int(blurred_image.shape[0]*size/100)), interpolation=cv2.INTER_CUBIC)
    return HR_img


def DN(image, size, noise_scale):
    """
    Dlown sampled by param size by BICUBIC interpolation and then Gausssian noise of given noise_scale is applied
    :param image:
    :param size:
    :param noise_scale:
    :return:
    """
    img = cv2.resize(image,
                        (int(image.shape[1] * size / 100), int(image.shape[0] * size / 100)),
                        interpolation=cv2.INTER_CUBIC)
    noise = np.random.normal(scale=noise_scale, size=img.shape)
    noise = noise.round()
    noisy_image = img.astype(np.int16) + noise.astype(np.int16)
    clipped_image = noisy_image.clip(0, 255).astype(np.uint8)
    return clipped_image


image = cv2.imread("kiwi.jpg")
cv2.imshow("original", image)
cv2.imshow("BD", BD(image, 50))
cv2.imshow("DN", DN(image, 50, 30))
cv2.waitKey(0)